In [3]:
#import all the modeules we've used before

import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot as plt
import seaborn as sns

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import rdkit
# Render the figure in a notebook:
%matplotlib inline  

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score

In [4]:
df = pd.read_csv('tox21.csv')
df_dropped_Al = df.drop([1322, 2290,2297,3558, 4565,4649, 5538, 6723, 7830])

df_only_data = df_dropped_Al.drop(columns=['mol_id', 'smiles']) #drop the columns.
df_only_data

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7825,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0
7826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
7827,1.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [5]:
X_data = pd.read_csv('dropped_Al_desc.csv')
X_data

,Unnamed: 0,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,smiles,mol_id
0,0,11.107593,11.107593,0.073011,-3.723219,0.900283,11.875000,258.324,248.244,258.013284,...,0,0,0,1,0,0,0,0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,TOX3021
1,1,11.790004,11.790004,0.172500,-0.515139,0.738013,20.600000,204.229,192.133,204.089878,...,0,0,0,0,0,0,0,1,CCN1C(=O)NC(c2ccccc2)C1=O,TOX3020
2,2,11.165775,11.165775,0.207144,-0.369724,0.666269,52.619048,288.475,256.219,288.245316,...,0,0,0,0,0,0,0,0,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...,TOX3024
3,3,12.570517,12.570517,0.042633,-0.042633,0.822261,12.500000,276.424,248.200,276.220164,...,0,0,0,0,0,0,0,0,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,TOX3027
4,4,10.251875,10.251875,0.383488,-5.197724,0.365583,15.090909,206.027,197.963,205.974526,...,0,0,0,0,0,0,0,0,CC(O)(P(=O)(O)O)P(=O)(O)O,TOX20800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7817,7817,12.539546,12.539546,0.053751,-0.598123,0.486536,11.920000,341.451,314.235,341.199094,...,0,0,0,0,0,0,0,0,CCCn1cnc2c1c(=O)n(CCCCC(C)=O)c(=O)n2C,TOX25189
7818,7818,11.819107,11.819107,0.191822,-1.001242,0.392116,11.060606,440.463,420.303,440.159689,...,0,0,1,0,0,0,0,0,c1cnc(N2CCN(Cc3ccc4c(c3)OCO4)CC2)nc1,TOX25188
7819,7819,12.146718,12.146718,0.245103,0.245103,0.701319,49.000000,314.469,284.229,314.224580,...,0,0,0,0,0,0,0,0,Cc1ccc(=O)n(-c2ccccc2)c1,TOX25183
7820,7820,11.787348,11.787348,0.082288,-0.082288,0.734230,52.714286,288.431,260.207,288.208930,...,0,0,0,0,0,0,0,0,Cc1nc2ccccn2c(=O)c1CCN1CCC(C(=O)c2ccc(F)cc2)CC1,TOX25182


In [6]:
x_train_val, x_test, y_train_val, y_test = train_test_split(X_data, df_only_data,test_size=0.15, random_state=0) #split the test off first
X_train, X_val, y_train, y_val = train_test_split(x_train_val,y_train_val,test_size = 0.15/0.85, random_state=0)

In [7]:
X_train

,Unnamed: 0,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,smiles,mol_id
1598,1598,9.814236,9.814236,0.031250,-0.031250,0.566800,53.454545,154.253,136.109,154.135765,...,0,0,0,0,0,0,0,0,CC(=O)O[C@@H]1C[C@H]2CC[C@]1(C)C2(C)C,TOX22061
3563,3563,11.902315,11.902315,0.254005,-1.688251,0.323312,29.812500,300.594,292.530,298.934133,...,0,0,0,0,0,0,0,0,C=C(C)C(=O)NCCC[N+](C)(C)C,TOX24682
3252,3252,2.120370,2.120370,1.310185,1.310185,0.511431,10.000000,128.174,120.110,128.062600,...,0,0,0,0,0,0,0,0,Nc1nc(N)nc(-c2cc(Cl)ccc2Cl)n1,TOX20234
5422,5422,11.767095,11.767095,0.165938,-1.392082,0.527270,30.200000,349.137,336.033,347.995698,...,0,0,0,0,0,0,0,0,CCOC(=O)C(C#N)=C(c1ccccc1)c1ccccc1,TOX24821
7747,7747,11.929391,11.929391,0.229761,-2.911845,0.515870,11.666667,373.371,353.211,373.086128,...,0,0,0,0,0,0,0,0,O=CCCl,TOX292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5501,5501,11.688537,11.688537,0.044753,-0.044753,0.122944,10.866667,426.682,376.282,426.370910,...,0,0,0,0,0,0,15,0,Nc1cc([N+](=O)[O-])ccc1C(=O)O,TOX963
1877,1877,11.799684,11.799684,0.298750,-0.702503,0.809002,10.687500,256.132,245.044,255.021769,...,0,1,0,0,0,0,0,0,Cl[C@H]1OCCO[C@@H]1Cl,TOX421
1699,1699,5.901944,5.901944,0.556571,-2.428721,0.482905,30.157895,288.460,260.236,288.175686,...,0,0,0,0,0,0,0,0,Cl[Nd](Cl)Cl,TOX24760
7282,7282,6.534497,6.534497,0.287913,-1.119655,0.424579,22.916667,532.472,501.224,531.180395,...,0,0,0,0,0,0,0,0,CN(C)c1ccc(O)c2c1C[C@H]1C[C@H]3[C@H](N(C)C)C(O...,TOX24545


In [8]:
y_train

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
1599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3567,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,0.0,1.0,1.0,1.0
3255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
7755,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7290,0.0,0.0,1.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0


In [9]:
df_NaN_1_train = y_train.applymap(lambda x: 1 if pd.isna(x) else x)  #   .applymap applies a lambda function to all values in the df that are NaN (defined by pf.isna(x))
df_NaN_1_val = y_val.applymap(lambda x: 1 if pd.isna(x) else x)
df_NaN_1_test = y_test.applymap(lambda x: 1 if pd.isna(x) else x)
df_NaN_1_val

C:\Users\nn21147\AppData\Local\Temp\ipykernel_8196\1821464590.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_NaN_1_train = y_train.applymap(lambda x: 1 if pd.isna(x) else x)  #   .applymap applies a lambda function to all values in the df that are NaN (defined by pf.isna(x))
C:\Users\nn21147\AppData\Local\Temp\ipykernel_8196\1821464590.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_NaN_1_val = y_val.applymap(lambda x: 1 if pd.isna(x) else x)
C:\Users\nn21147\AppData\Local\Temp\ipykernel_8196\1821464590.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_NaN_1_test = y_test.applymap(lambda x: 1 if pd.isna(x) else x)


,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
7099,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
5314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6048,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4246,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4169,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [35]:
df_NaN_0_train = y_train.applymap(lambda x: 0 if pd.isna(x) else x)  #   .applymap applies a lambda function to all values in the df that are NaN (defined by pf.isna(x))
df_NaN_0_val = y_val.applymap(lambda x: 0 if pd.isna(x) else x)
df_NaN_0_test = y_test.applymap(lambda x: 0 if pd.isna(x) else x)
df_NaN_0_train

C:\Users\nn21147\AppData\Local\Temp\ipykernel_8196\1046230712.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_NaN_0_train = y_train.applymap(lambda x: 0 if pd.isna(x) else x)  #   .applymap applies a lambda function to all values in the df that are NaN (defined by pf.isna(x))
C:\Users\nn21147\AppData\Local\Temp\ipykernel_8196\1046230712.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_NaN_0_val = y_val.applymap(lambda x: 0 if pd.isna(x) else x)
C:\Users\nn21147\AppData\Local\Temp\ipykernel_8196\1046230712.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_NaN_0_test = y_test.applymap(lambda x: 0 if pd.isna(x) else x)


,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
1599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
3255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7755,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7290,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

toxics = []

for row in df_NaN_0_train.index: #select each row
    #print(row)
     istoxic = False
     for i in df_NaN_0_train.iloc[row]: #select each row
          # print(i,type(i))
          if i == 1.0:
             istoxic = True
     if istoxic == True:
          toxics.append(row)

IndexError: single positional indexer is out-of-bounds